In [1]:
import cv2
import numpy as np
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
import imageio

from moviepy.editor import VideoFileClip
from IPython.display import HTML

%matplotlib inline

class Line():
    def __init__(self):
        self.fit = []
        self.fit = []
        self.n =0
        self.fit_mean = []
        self.fit_mean = []
        
### CAMERA CALIBRATION ###----------------------------------------------------
def cal_undistort(img, objpoints, imgpoints):
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1],None,None)
        undist = cv2.undistort(img, mtx,dist,None,mtx)
        return undist, mtx,dist
def Camera_Calibration(glob_import):
    objp = np.zeros((6*9,3), np.float32)
    objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)
    
    objpoints = []
    imgpoints = []

    images = glob.glob(glob_import)

    # Step through the list and search for chessboard corners
    for fname in images:
        img = cv2.imread(fname)
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

        # Find the chessboard corners
        ret, corners = cv2.findChessboardCorners(gray, (9,6),None)

        # If found, add object points, image points
        if ret == True:
            objpoints.append(objp)
            imgpoints.append(corners)

            undistorted,mtx,dist = cal_undistort(img, objpoints, imgpoints)

    return mtx,dist


## CREATING A THRESHOLDED BINARY IMAGE ### ------------------------------------
def binary_img(img,s_thresh=(100, 255), sx_thresh=(20, 100)):
    
    img = np.copy(img)
    # Convert to HLS color space and separate the V channel
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]
    # Sobel x
    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) # Take the derivative in x
    abs_sobelx = np.absolute(sobelx) # Absolute x derivative to accentuate lines away from horizontal
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    # Threshold x gradient
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    # Threshold color channel
    s_binary = np.zeros_like(scaled_sobel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1
    # Stack each channel
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[((s_binary == 1) | (sxbinary == 1))] = 1
    
    return combined_binary

def abs_sobel_thresh(img, orient='x', sobel_kernel=3, thresh=(0, 255)):
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    if orient =='x':
        sobel = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    else:
        sobel = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    # 3) Take the absolute value of the derivative or gradient
    abs_sobel = np.absolute(sobel)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobel/np.max(abs_sobel))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
    grad_binary = np.zeros_like(scaled_sobel)
    grad_binary[(scaled_sobel>=thresh[0]) & (scaled_sobel<=thresh[1])] = 1
    return grad_binary

def mag_thresh(img, sobel_kernel=3, mag_thresh=(0, 255)):
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # 2) Take the derivative in x or y given orient = 'x' or 'y'
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely = cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    # 3) Calculate the magnitude 
    abs_sobelxy = np.sqrt(sobelx**2+sobely**2)
    # 4) Scale to 8-bit (0 - 255) then convert to type = np.uint8
    scaled_sobel = np.uint8(255*abs_sobelxy/np.max(abs_sobelxy))
    # 5) Create a mask of 1's where the scaled gradient magnitude 
    mag_binary = np.zeros_like(scaled_sobel)
    mag_binary[(scaled_sobel>mag_thresh[0]) & (scaled_sobel<mag_thresh[1])] = 1
    # 6) Return this mask as your binary_output image
    return mag_binary

def dir_threshold(img, sobel_kernel=3, thresh=(0, np.pi/2)):
    # 1) Convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_RGB2GRAY)
    # 2) Take the gradient in x and y separately
    sobelx = cv2.Sobel(gray,cv2.CV_64F,1,0,ksize = sobel_kernel)
    sobely= cv2.Sobel(gray,cv2.CV_64F,0,1,ksize = sobel_kernel)
    # 3) Take the absolute value of the x and y gradients
    abs_sobelx= np.absolute(sobelx)
    abs_sobely= np.absolute(sobely)
    # 4) Use np.arctan2(abs_sobely, abs_sobelx) to calculate the direction of the gradient 
    tg_sobel = np.arctan2(abs_sobely,abs_sobelx)
    # 5) Create a binary mask where direction thresholds are met
    dir_binary = np.zeros_like(tg_sobel)
    dir_binary[(tg_sobel>=thresh[0]) & (tg_sobel<=thresh[1])] = 1

    return dir_binary  
    
## BIRD EYE VIEW ##-----------------------------------------------------------
def bird_eye_view(img):
    
    img_size = (img.shape[1], img.shape[0])
    offset = 100   
    
    src = np.float32([[580.815,460.748],[707.973,460.748],[243.095,691.82],[1059.37,691.82]]) # By image
 
    dst = np.float32([[320,0],[960,0],[320,720],[960,720]])

    M = cv2.getPerspectiveTransform(src,dst)
    img_size = (img.shape[1],img.shape[0])
    warped = cv2.warpPerspective(img,M,img_size, flags = cv2.INTER_LINEAR)
    
    return warped,M

    
## Detecting Lane-------------------------------------------------------------
def find_lane_pixels(binary_warped):
    # Take a histogram of the bottom half of the image
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis=0)
    # Create an output image to draw on and visualize the result
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    # Find the peak of the left and right halves of the histogram
    # These will be the starting point for the left and right lines
    midpoint = np.int(histogram.shape[0]//2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint

    # Choose the number of sliding windows
    nwindows = 9
    # Set the width of the windows +/- margin
    margin = 100
    # Set minimum number of pixels found to recenter window
    minpix = 50
    # Set height of windows - based on nwindows above and image shape
    window_height = np.int(binary_warped.shape[0]//nwindows)
    # Identify the x and y positions of all nonzero pixels in the image
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    # Current positions to be updated later for each window in nwindows
    leftx_current = leftx_base
    rightx_current = rightx_base

    # Create empty lists to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    for window in range(nwindows):
        
        # Identify window boundaries in x and y (and right and left)
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height

        win_xleft_low = leftx_current - margin 
        win_xleft_high = leftx_current + margin 
        win_xright_low = rightx_current - margin  
        win_xright_high = rightx_current + margin  
        
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low),
        (win_xleft_high,win_y_high),(0,255,0), 2) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low),
        (win_xright_high,win_y_high),(0,255,0), 2) 
        
        good_left_inds = ((nonzeroy >= win_y_low)&(nonzeroy<win_y_high)&
                         (nonzerox >= win_xleft_low)&(nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low)&(nonzeroy<win_y_high)&
                         (nonzerox >= win_xright_low)&(nonzerox < win_xright_high)).nonzero()[0]
        
        
        # Append these indices to the lists
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds)> minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds)> minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

    # Concatenate the arrays of indices (previously was a list of lists of pixels)
    try:
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)
    except ValueError:
        # Avoids an error if the above is not implemented fully
        pass

    # Extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def fit_poly(img_shape, leftx, lefty, rightx, righty):
    left_fit = left.fit
    right_fit = right.fit
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, img_shape[0]-1, img_shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
    return left_fitx, right_fitx, ploty

def search_around_poly(undist,M,binary_warped):
    margin = 70
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    left_fit = np.polyfit(lefty,leftx,2)
    right_fit = np.polyfit(righty,rightx,2)
    
    if left.fit == []:
        left.fit = left_fit
        right.fit = right_fit
        
    else:
        
        if (left_fit[1]*right_fit[1]>0)and(abs(left_fit[0]-right_fit[0])<0.001):

            if left.n<3:
                left_fit.tolist()
                right_fit.tolist()

                left.fit_mean.append(left_fit) 
                right.fit_mean.append(right_fit) 

                left_fit = left.fit
                right_fit= right.fit

                left.n +=1
            else:

                left.fit_mean = np.array(left.fit_mean).mean(0)
                right.fit_mean = np.array(right.fit_mean).mean(0)
                # UPDATE
                left.fit = left.fit_mean
                right.fit = right.fit_mean
                
                left.fit_mean =[]
                right.fit_mean = []

                left_fit = left.fit
                right_fit= right.fit

                left.n = 0
        else:   
            left_fit = left.fit
            right_fit= right.fit 

    # Grab activated pixels
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    left_lane_inds = ((nonzerox >(left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] -margin))
                     &(nonzerox <(left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] +margin)))
    right_lane_inds = ((nonzerox >(right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] -margin))
                     &(nonzerox <(right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] +margin)))

    # Again, extract left and right line pixel positions
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds] 
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    
    left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)

    ## Visualization ##******************************************
    # Create an image to draw on and an image to show the selection window
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    # Color in left and right line pixels
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255, 0, 0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0, 0, 255]

    # Generate a polygon to illustrate the search window area
    # And recast the x and y points into usable format for cv2.fillPoly()
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)


    ## End visualization steps ##
    left_curverad,right_curverad,left_curverad_pix, diff = measure_curvature_real(ploty,left_fitx,right_fitx)

    # Drawing
    final_image = drawing(binary_warped,undist,left_fitx, right_fitx, ploty,M)


    return result, left_curverad, right_curverad , final_image,left_curverad_pix, diff
  
def measure_curvature_real(ploty,left_fitx,right_fitx):
    ym_per_pix = 30/720 # meters per pixel in y dimension
    xm_per_pix = 3.7/600 # meters per pixel in x dimension
    
    left_pixel = left_fitx
    right_pixel = right_fitx
    
    ploty = ploty*ym_per_pix
    left_fitx = left_fitx*xm_per_pix
    right_fitx = right_fitx*xm_per_pix
    
    # left and right for pixel
    left_fit_cr = np.polyfit(ploty, left_pixel, 2)
    right_fit_cr = np.polyfit(ploty, right_pixel, 2)  
    y_eval = np.max(ploty)
    left_curverad_pix = ((1+(2*left_fit_cr[0]*y_eval+left_fit_cr[1])**2)**(3/2))/np.absolute(2*left_fit_cr[0])  ## Implement the calculation of the left line here
    right_curverad_pix = ((1+(2*right_fit_cr[0]*y_eval+right_fit_cr[1])**2)**(3/2))/np.absolute(2*right_fit_cr[0])  ## Implement the calculation of the right line here
    diff = abs(left_curverad_pix-right_curverad_pix)
    
    #left and rightt for real curvature
    left_fit_cr = np.polyfit(ploty, left_fitx, 2)
    right_fit_cr = np.polyfit(ploty, right_fitx, 2)  
    y_eval = np.max(ploty)
    left_curverad = ((1+(2*left_fit_cr[0]*y_eval+left_fit_cr[1])**2)**(3/2))/np.absolute(2*left_fit_cr[0])  ## Implement the calculation of the left line here
    right_curverad = ((1+(2*right_fit_cr[0]*y_eval+right_fit_cr[1])**2)**(3/2))/np.absolute(2*right_fit_cr[0])  ## Implement the calculation of the right line here
    
    
    return left_curverad, right_curverad, left_curverad_pix, diff

### Drawing ###-----------------------------------------------------------------

def drawing(warped,undist,left_fitx,right_fitx,ploty,M):
    
    warp_zero = np.zeros_like(warped).astype(np.uint8)
    color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))
    Minv = np.linalg.inv(M)
    
    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    newwarp = cv2.warpPerspective(color_warp, Minv, (undist.shape[1], undist.shape[0])) 
    # Combine the result with the original image
    result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
    
#     plt.imshow(result)
#     plt.show()
    return result


#Main ------------------------------------------------------------------------   
def process_image(image_initial):
    
    image_initial = cv2.resize(image_initial,(1280,720))
    
    ## Distortion Correction
    undist = cv2.undistort(image_initial,mtx,dist,None,mtx)
    
    ## Binary
    color_binary = binary_img(undist)
    ksize = 3
    gradx = abs_sobel_thresh(undist, orient='x', sobel_kernel=ksize, thresh=(20, 100))
    grady = abs_sobel_thresh(undist, orient='y', sobel_kernel=ksize, thresh=(20, 100))
    mag_binary = mag_thresh(undist, sobel_kernel=ksize, mag_thresh=(30, 100))
    dir_binary = dir_threshold(undist, sobel_kernel=ksize, thresh=(0.7, 1.3))
    combined = np.zeros_like(dir_binary)
    combined[((gradx == 1) & (grady == 1)) | ((mag_binary == 1) & (dir_binary == 1))] = 1
    combined_plus = np.zeros_like(color_binary)
    combined_plus[((color_binary == 1) | (combined == 1))]= 1
 
    # Bird Eye View
    warped,M = bird_eye_view(combined_plus)
    warped[:,:150] = 0
    
    ## Detecting Lane
    result, lf,righ, final_image,left_curverad_pix, diff = search_around_poly(undist,M,warped)
    
    ##  IMAGE PLOTS ##
#     plt.imshow(image_initial)
#     plt.show()
#     plt.imshow(color_binary,'gray')
#     plt.show()
#     plt.imshow(combined,'gray')
#     plt.show()
    
#     plt.imshow(warped,'gray')
#     plt.show()
    
#     plt.imshow(result)
#     plt.show()

### WRITE IN IMAGE ###-------------------------------------------------------------------------------------------------------
    xm_per_pix = 3.7/600 # meters per pixel in x dimension
    
    
    font = cv2.FONT_HERSHEY_SIMPLEX
    
    radius = int((lf+righ)/2)
    
    text = 'Radius of Curvature = '+ str(radius)+'(m)'
    cv2.putText(final_image,text,(150,100),font,2,(255,255,255),2,cv2.LINE_AA)
    
    of_center = round(abs(left_curverad_pix-diff)*xm_per_pix,2)
    text2 = 'Vehicle is '+ str(of_center)+'(m) left of center'
    cv2.putText(final_image,text2,(150,150),font,1.8,(255,255,255),2,cv2.LINE_AA)
    
    return final_image


global mtx,dist
glob_import = 'CarND-Advanced-Lane-Lines-master/camera_cal/calibration*.jpg'      
mtx,dist = Camera_Calibration(glob_import)
left = Line()
right = Line()

white_output = 'CarND-Advanced-Lane-Lines-master/output_tests/Video_Final.mp4'

clip1 = VideoFileClip('CarND-Advanced-Lane-Lines-master/project_video.mp4')

white_clip = clip1.fl_image(process_image)
%time white_clip.write_videofile(white_output, audio = False)
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(white_output))


TypeError: src is not a numpy array, neither a scalar